In [15]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('../data/0927到1022.csv',index_col=0)
df.head()

,項次,電文代號,銀行,20,回傳代碼,回傳訊息,建檔日期,受益人,金額,推薦公司事業部,...,MEMO,from,45A,50,59,46A,47A,78,memo,Unnamed: 5
7,8,mt700,16,0006MLC00034022,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:31:17,FORMOSA PLASTICS CORPORATION\n201 TUNG HWA NOR...,"USD136080,00",11,...,NaN,0927.xls,48 MT PVC SUSPENSION RESIN GRADE S-65\r\nAT TH...,DEEP JYOTI WAX TRADERS\r\nPRIVATE LIMITED\r\n1...,FORMOSA PLASTICS CORPORATION\r\n201 TUNG HWA N...,1. SIGNED COMMERCIAL INVOICE IN TWO ORIGINAL.\...,1. ALL DOCUMENTS MUST BE IN ENGLISH.\r\n2. ALL...,UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT ...,NaN,NaN
9,10,mt700,16,001LM01212660007,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:32:17,NAN YA PLASTICS CORPORATION\nREFER FIELD 47A C...,"USD111000,",24,...,NaN,0927.xls,TETRAHYDROFURAN AT USD 5550 TOTAL AMOUNT-USD 1...,UJIN PHARMACHEM\r\nREFER FIELD 47A POINT NO.12,NAN YA PLASTICS CORPORATION\r\nREFER FIELD 47A...,+1.SIGNED COMMERCIAL INVOICES IN 6 COPIES FOR ...,+1.ALL DOCUMENT MUST BE IN ENGLISH LANGUAGE.\r...,"IN REIMBURSEMENT,WE SHALL REMIT PROCEEDS ON TH...",NaN,NaN
10,11,mt700,16,0393MLC00068522,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:33:05,"FORMOSA CHEMICALS AND FIBRE\nCORPORATION, 201,...","USD736110,00",4A,...,NaN,0927.xls,A) 198.00 M.T. OF TAIRILAC AG15A1-H(ABS) ABS ...,SUPREME PETROCHEM LIMITED\r\nREFER FIELD 47A C...,"FORMOSA CHEMICALS AND FIBRE\r\nCORPORATION, 20...","1.SIGNED COMMERCIAL INVOICE IN 3 COPIES, CER...",1.ALL DOCUMENTS MUST BE IN ENGLISH.\r\n2. ALL ...,+ NEGOTIATING BANK CAN CLAIM REIMBURSEMENT ON ...,NaN,NaN
11,12,mt700,16,0505121IM0000553,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:01:28,"FORMOSA PLASTICS CORPORATION\n201 TUNG HWA, NO...","USD154710,00",11,...,NaN,0927.xls,80 MT PVC SUSPENSION RESIN GRADE S-65S AT THE ...,"RMG POLYVINYL INDIA LIMITED\r\n10/1, INDUSTRIA...","FORMOSA PLASTICS CORPORATION\r\n201 TUNG HWA, ...",1. DRAFT FOR 100 PERCENT OF THE INVOICE VALUE....,1.INLAND HAULAGE CHARGES FROM NHAVA SHEVA PORT...,1. WE WILL REMIT THE PROCEEDS TO THE NEGOTIATI...,NaN,NaN
12,13,mt700,16,0799M21SA0000380,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:01:04,"FORMOSA PLASTICS CORPORATION\n201, TUNG HWA N....","USD152640,",11,...,NaN,0927.xls,"PVC SUSPENSION RESIN S-70R\r\nQUANTITY: 72 MT,...",KTM KIMYEVI MADDELER ITHALAT\r\nIHRACAT SANAYI...,"FORMOSA PLASTICS CORPORATION\r\n201, TUNG HWA ...",1. COMMERCIAL INVOICE IN 2 ORIGINALS AND 2 COP...,UNLESS OTHERWISE STIPULATED IN THE CREDIT\r\n1...,UPON OUR RECEIPT OF STRICTLY COMPLYING DOCUMEN...,NaN,NaN


In [16]:
for fname in tqdm(sorted([ i for i in df['from'].unique()])):
    d = df.loc[df['from']==fname]
    save_name = '../data/測試結果/'+fname[:4]+'測試結果'
    d.to_csv(save_name)
    print(save_name,d.shape)
    

100%|██████████| 15/15 [00:00<00:00, 132.62it/s]

../data/測試結果/0927測試結果 (34, 21)
../data/測試結果/0928測試結果 (56, 21)
../data/測試結果/0929測試結果 (50, 21)
../data/測試結果/1004測試結果 (34, 21)
../data/測試結果/1005測試結果 (30, 21)
../data/測試結果/1006測試結果 (41, 21)
../data/測試結果/1007測試結果 (36, 21)
../data/測試結果/1008測試結果 (31, 21)
../data/測試結果/1012測試結果 (35, 21)
../data/測試結果/1013測試結果 (43, 21)
../data/測試結果/1018測試結果 (39, 21)
../data/測試結果/1019測試結果 (26, 21)
../data/測試結果/1020測試結果 (45, 21)
../data/測試結果/1021測試結果 (36, 21)
../data/測試結果/1022測試結果 (52, 21)
